In [2]:
class AVLNode:
    def __init__(self, key):
        self.key = key
        self.left = None
        self.right = None
        self.height = 1  # Height of the node (leaf node has height 1)

class AVLTree:
    def __init__(self):
        self.root = None

    # Helper function to get height of a node
    def get_height(self, node):
        if not node:
            return 0
        return node.height

    # Helper function to get balance factor of a node
    def get_balance(self, node):
        if not node:
            return 0
        return self.get_height(node.left) - self.get_height(node.right)

    # Helper function to update height of a node
    def update_height(self, node):
        if not node:
            return
        node.height = 1 + max(self.get_height(node.left), self.get_height(node.right))

    # Right rotation
    def right_rotate(self, y):
        x = y.left
        T2 = x.right

        # Perform rotation
        x.right = y
        y.left = T2

        # Update heights
        self.update_height(y)
        self.update_height(x)

        # Return new root
        return x

    # Left rotation
    def left_rotate(self, x):
        y = x.right
        T2 = y.left

        # Perform rotation
        y.left = x
        x.right = T2

        # Update heights
        self.update_height(x)
        self.update_height(y)

        # Return new root
        return y

    # Insert a node
    def insert(self, key):
        self.root = self._insert(self.root, key)

    def _insert(self, node, key):
        # Perform standard BST insert
        if not node:
            return AVLNode(key)

        if key < node.key:
            node.left = self._insert(node.left, key)
        elif key > node.key:
            node.right = self._insert(node.right, key)
        else:
            # Duplicate keys not allowed
            return node

        # Update height of current node
        self.update_height(node)

        # Get balance factor
        balance = self.get_balance(node)

        # Left Left Case
        if balance > 1 and key < node.left.key:
            return self.right_rotate(node)

        # Right Right Case
        if balance < -1 and key > node.right.key:
            return self.left_rotate(node)

        # Left Right Case
        if balance > 1 and key > node.left.key:
            node.left = self.left_rotate(node.left)
            return self.right_rotate(node)

        # Right Left Case
        if balance < -1 and key < node.right.key:
            node.right = self.right_rotate(node.right)
            return self.left_rotate(node)

        # Return the unchanged node pointer
        return node

    # Find minimum value node in a subtree
    def get_min_value_node(self, node):
        current = node
        while current.left:
            current = current.left
        return current

    # Delete a node
    def delete(self, key):
        self.root = self._delete(self.root, key)

    def _delete(self, root, key):
        # Perform standard BST delete
        if not root:
            return root

        if key < root.key:
            root.left = self._delete(root.left, key)
        elif key > root.key:
            root.right = self._delete(root.right, key)
        else:
            # Node with only one child or no child
            if not root.left:
                return root.right
            elif not root.right:
                return root.left

            # Node with two children
            # Get the inorder successor (smallest in the right subtree)
            temp = self.get_min_value_node(root.right)

            # Copy the inorder successor's content to this node
            root.key = temp.key

            # Delete the inorder successor
            root.right = self._delete(root.right, temp.key)

        # If the tree had only one node then return
        if not root:
            return root

        # Update height of the current node
        self.update_height(root)

        # Get the balance factor
        balance = self.get_balance(root)

        # If the node becomes unbalanced, then there are 4 cases

        # Left Left Case
        if balance > 1 and self.get_balance(root.left) >= 0:
            return self.right_rotate(root)

        # Left Right Case
        if balance > 1 and self.get_balance(root.left) < 0:
            root.left = self.left_rotate(root.left)
            return self.right_rotate(root)

        # Right Right Case
        if balance < -1 and self.get_balance(root.right) <= 0:
            return self.left_rotate(root)

        # Right Left Case
        if balance < -1 and self.get_balance(root.right) > 0:
            root.right = self.right_rotate(root.right)
            return self.left_rotate(root)

        return root

    # Search for a key
    def search(self, key):
        return self._search(self.root, key)

    def _search(self, root, key):
        if not root or root.key == key:
            return root

        if key < root.key:
            return self._search(root.left, key)
        return self._search(root.right, key)

    # In-order traversal
    def inorder(self):
        result = []
        self._inorder(self.root, result)
        return result

    def _inorder(self, root, result):
        if root:
            self._inorder(root.left, result)
            result.append(root.key)
            self._inorder(root.right, result)

    # Pre-order traversal
    def preorder(self):
        result = []
        self._preorder(self.root, result)
        return result

    def _preorder(self, root, result):
        if root:
            result.append(root.key)
            self._preorder(root.left, result)
            self._preorder(root.right, result)

    # Post-order traversal
    def postorder(self):
        result = []
        self._postorder(self.root, result)
        return result

    def _postorder(self, root, result):
        if root:
            self._postorder(root.left, result)
            self._postorder(root.right, result)
            result.append(root.key)

    # Print the tree
    def display(self):
        lines, *_ = self._display_aux(self.root)
        for line in lines:
            print(line)

    def _display_aux(self, node):
        """Returns list of strings, width, height, and horizontal coordinate of the root."""
        # No child.
        if node.right is None and node.left is None:
            line = f"{node.key}({node.height})"
            width = len(line)
            height = 1
            middle = width // 2
            return [line], width, height, middle

        # Only left child.
        if node.right is None:
            lines, n, p, x = self._display_aux(node.left)
            s = f"{node.key}({node.height})"
            u = len(s)
            first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s
            second_line = x * ' ' + '/' + (n - x - 1 + u) * ' '
            shifted_lines = [line + u * ' ' for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, n + u // 2

        # Only right child.
        if node.left is None:
            lines, n, p, x = self._display_aux(node.right)
            s = f"{node.key}({node.height})"
            u = len(s)
            first_line = s + x * '_' + (n - x) * ' '
            second_line = (u + x) * ' ' + '\\' + (n - x - 1) * ' '
            shifted_lines = [u * ' ' + line for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, u // 2

        # Two children.
        left, n, p, x = self._display_aux(node.left)
        right, m, q, y = self._display_aux(node.right)
        s = f"{node.key}({node.height})"
        u = len(s)
        first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s + y * '_' + (m - y) * ' '
        second_line = x * ' ' + '/' + (n - x - 1 + u + y) * ' ' + '\\' + (m - y - 1) * ' '
        if p < q:
            left += [n * ' '] * (q - p)
        elif q < p:
            right += [m * ' '] * (p - q)
        zipped_lines = zip(left, right)
        lines = [first_line, second_line] + [a + u * ' ' + b for a, b in zipped_lines]
        return lines, n + m + u, max(p, q) + 2, n + u // 2

# Test the AVL Tree implementation
def test_avl_tree():
    # Test case 1: Insertion and balanced tree
    avl = AVLTree()

    # Insert elements and test balance
    test_data = [10, 20, 30, 40, 50, 25, 15]
    print("Inserting:", test_data)
    for key in test_data:
        avl.insert(key)

    print("\nAVL Tree after insertions:")
    avl.display()

    print("\nIn-order traversal (should be sorted):", avl.inorder())
    print("Pre-order traversal:", avl.preorder())
    print("Post-order traversal:", avl.postorder())

    # Test search
    print("\nTesting search functionality:")
    search_keys = [30, 100]
    for key in search_keys:
        result = avl.search(key)
        if result:
            print(f"Key {key} found in the tree")
        else:
            print(f"Key {key} not found in the tree")

    # Test case 2: Deletion and re-balancing
    print("\nTesting deletion functionality:")
    delete_keys = [20, 30, 50]
    for key in delete_keys:
        print(f"Deleting key {key}")
        avl.delete(key)
        print(f"Tree after deleting {key}:")
        avl.display()
        print("In-order traversal:", avl.inorder())

    # Test case 3: Creating a new tree with a different pattern
    print("\nTest case 3: Creating a new AVL tree with different pattern")
    avl2 = AVLTree()
    test_data2 = [100, 50, 150, 25, 75, 125, 175, 12, 37, 63, 87]
    for key in test_data2:
        avl2.insert(key)

    print("\nAVL Tree 2 after insertions:")
    avl2.display()
    print("In-order traversal:", avl2.inorder())

    print("\nDeleting keys 100 (root) and 50:")
    avl2.delete(100)
    avl2.delete(50)

    print("AVL Tree 2 after deletions:")
    avl2.display()
    print("In-order traversal:", avl2.inorder())

if __name__ == "__main__":
    test_avl_tree()

Inserting: [10, 20, 30, 40, 50, 25, 15]

AVL Tree after insertions:
             _______30(4)__        
            /              \       
   _______20(3)__        40(2)__   
  /              \              \  
10(2)__        25(1)          50(1)
       \                           
     15(1)                         

In-order traversal (should be sorted): [10, 15, 20, 25, 30, 40, 50]
Pre-order traversal: [30, 20, 10, 15, 25, 40, 50]
Post-order traversal: [15, 10, 25, 20, 50, 40, 30]

Testing search functionality:
Key 30 found in the tree
Key 100 not found in the tree

Testing deletion functionality:
Deleting key 20
Tree after deleting 20:
        _______30(3)__        
       /              \       
   __15(2)__        40(2)__   
  /         \              \  
10(1)     25(1)          50(1)
In-order traversal: [10, 15, 25, 30, 40, 50]
Deleting key 30
Tree after deleting 30:
        _______40(3)__   
       /              \  
   __15(2)__        50(1)
  /         \            
10(1)  